In [726]:
import math
import numpy as np

In [ ]:
class Neuron:
    def __init__(self, value, _children=(), _op='', label=''):
        self.value = value
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children);
        self._op = _op;
        self.label = label # for visualization

    def __repr__(self):
        stringVal = f"{self.value}";
        return stringVal

    def __add__(self,other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        out = Neuron(self.value + other.value, (self, other), '+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __mul__(self,other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        out = Neuron(self.value * other.value, (self, other), '*')
        def _backward():
            self.grad += other.value * out.grad
            other.grad += self.value * out.grad
        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers"
        out = Neuron(self.value**other, (self, ), f'**{other}')
        def _backward():
            self.grad += other * (self.value ** (other - 1)) * out.grad
        out._backward = _backward
        return out

    def __truediv__(self, other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        return self * other**-1

    def exp(self, pos=1):
        x = self.value*pos
        out = Neuron(math.exp(x), (self, ), 'exp')
        def _backward():
            self.grad += out.value * out.grad
        out._backward = _backward
        return out

    def __radd__(self, other): # other + self
        return self + other

    def __rsub__(self, other): # other - self
        return -(self - other)

    def __rmul__(self, other): # other * self
        return self * other

    def __rtruediv__(self, other): # other / self
        return self / other**-1

    def backward(self):
        topology = []
        visited = set()
        def build_topology(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topology(child)
                topology.append(v)
        build_topology(self)

        self.grad = 1.0
        for node in reversed(topology):
            node._backward()

    # Activation function
    def sigmoid(self):
        x  = self.value
        s = 1/(1+math.exp(-x))
        out = Neuron(s, (self, ), "sigmoid")

        def _backward():
            self.grad += (s*(1-s)) * out.grad
        out._backward = _backward

        return out

    def linear(self):
        out = Neuron(self.value, (self, ), "linear")

        def _backward():
            self.grad += 1*out.grad
        out._backward = _backward

        return out
    
    def relu(self):
        x = self.value
        r = x if x > 0 else 0
        dr = 1 if x > 0 else 0
        out = Neuron(r, (self, ), "relu")

        def _backward():
            self.grad += dr*out.grad
        out._backward = _backward

        return out
    
    def tanh(self):
        x = self.value
        t = (math.tanh(x))
        out = Neuron(t, (self, ), "tanh")

        def _backward():
            self.grad += (1-t**2) * out.grad
        out._backward = _backward

        return out
    
    def softmax(self, sum):
        return self.exp()/sum
    
    def swish(self):
        x = self.value
        sigmoid = 1/(1+math.exp(-x))
        swish = x*sigmoid
        out = Neuron(swish, (self, ), "swish")

        def _backward():
            self.grad += (sigmoid + x*sigmoid*(1-sigmoid)) * out.grad
        out._backward = _backward

        return out
    
    def leakyrelu(self, alpha=0.01):
        x = self.value
        lr = x if x > 0 else alpha*x
        dlr = 1 if x > 0 else alpha
        out = Neuron(lr, (self, ), "leakyrelu")

        def _backward():
            self.grad +=  dlr*out.grad
        out._backward = _backward

        return out

    def hello():
        print("hello")

In [728]:
class FFNN19:
    def __init__(self,
                 hidden_layer_count=1,
                 input_neuron_count=1,
                 output_neuron_count=1,
                 activation_function="linear"
                 ):

        self.hidden_layer_count             = hidden_layer_count;
        self.input_neuron_count             = input_neuron_count;
        self.output_neuron_count            = output_neuron_count;
        self._weight_layer_count            = hidden_layer_count+1;
        self.activation_function            = activation_function
        self.weight_layers                  = [Neuron(0) for _ in range(self._weight_layer_count)];
        self.hidden_layers                  = [Neuron(0) for _ in range(hidden_layer_count)];
        self._activated_hidden_layers       = [Neuron(0) for _ in range(hidden_layer_count)];
        self._output_layer                  = [Neuron(0) for _ in range(output_neuron_count)];
        self._neuron_layer_set              = False;
        self._weight_layers_initialized     = False;
        self.outputs                        = []

    # Vectorize is not the most efficient solution (gw mager)
    def neuronGenerator(self,generator, *args, **kwargs):
        value = generator(*args, **kwargs)
        return np.vectorize(Neuron)(value)

    def setHiddenLayerNeuronCount(self,*args):
        if len(args)==0:
            for i in range(self.hidden_layer_count):
                self.hidden_layers[i] = self.neuronGenerator(np.zeros,(input())+1)
                self.hidden_layers[i][0] = Neuron(1)
            self._neuron_layer_set    = True
        elif len(args)==1:
            if len(args[0])!=self.hidden_layer_count:
                raise ValueError(f"Layer count mismatch: expected {self.hidden_layer_count} hidden layer, but {len(args[0])} were given")
            else:
                for i in range(self.hidden_layer_count):
                    self.hidden_layers[i] = self.neuronGenerator(np.zeros,args[0][i]+1)
                    self.hidden_layers[i][0] = Neuron(1)
                self._neuron_layer_set = True
        else:
            raise TypeError(f"FFNN19.setHiddenLayerNeuronCount() takes exactly 1 positional arguments but {len(args)} were given")

    def zeroWeightInitialization(self):
        if self._neuron_layer_set:
            for i in range(self._weight_layer_count):
                if i==0:
                    self.weight_layers[i] = self.neuronGenerator(np.zeros,(self.input_neuron_count+1,len(self.hidden_layers[i])))
                elif i==self._weight_layer_count-1:
                    self.weight_layers[i] = self.neuronGenerator(np.zeros,(len(self.hidden_layers[i-1]),self.output_neuron_count))
                else:
                    self.weight_layers[i] = self.neuronGenerator(np.zeros,(len(self.hidden_layers[i-1]),len(self.hidden_layers[i])))
            self._weight_layers_initialized     = True;
        else:
            raise ValueError(self._neuron_layer_set_message)

    def uniformWeightDistribution(self,lower,upper,seed):
        if self._neuron_layer_set:
            rng = np.random.default_rng(seed)
            for i in range(self._weight_layer_count):
                if i==0:
                    self.weight_layers[i] = self.neuronGenerator(rng.uniform,lower,upper,size=(self.input_neuron_count+1,len(self.hidden_layers[i])))
                elif i==self._weight_layer_count-1:
                    self.weight_layers[i] = self.neuronGenerator(rng.uniform,lower,upper,size=(len(self.hidden_layers[i-1]),self.output_neuron_count))
                else:
                    self.weight_layers[i] = self.neuronGenerator(rng.uniform,lower,upper,size=(len(self.hidden_layers[i-1]),len(self.hidden_layers[i])))
                if i!=self._weight_layer_count-1:
                    self.weight_layers[i][:,0] = Neuron(0)
            self._weight_layers_initialized     = True;
        else:
            raise ValueError(f"Neuron layer count not set, please set neuron layer count before weight initialization")

    def normalWeightDistribution(self,mean,variance,seed):
        if self._neuron_layer_set:
            rng = np.random.default_rng(seed)
            for i in range(self._weight_layer_count):
                if i==0:
                    self.weight_layers[i] = self.neuronGenerator(rng.normal,mean,variance,size=(self.input_neuron_count+1,len(self.hidden_layers[i])))
                elif i==self._weight_layer_count-1:
                    self.weight_layers[i] = self.neuronGenerator(rng.normal,mean,variance,size=(len(self.hidden_layers[i-1]),self.output_neuron_count))
                else:
                    self.weight_layers[i] = self.neuronGenerator(rng.normal,mean,variance,size=(len(self.hidden_layers[i-1]),len(self.hidden_layers[i])))
                if i!=self._weight_layer_count-1:
                    self.weight_layers[i][:,0] = Neuron(0)
            self._weight_layers_initialized     = True;
        else:
            raise ValueError(f"Neuron layer count not set, please set neuron layer count before weight initialization")

    # Single pass, masih bingung gmn caranya buat multiple
    def _feedforward(self,input_data):
        if self._weight_layers_initialized:
            input_data.insert(0,1)
            input_layer_neuron = self.neuronGenerator(np.array,object=input_data)
            for i in range(self._weight_layer_count):
                if i==0:
                    self.hidden_layers[i] = np.sum(input_layer_neuron*self.weight_layers[i].T,axis=1)
                    # self.hidden_layers[i] = np.vectorize(lambda n: n.tanh())(self.hidden_layers[i])
                    self.hidden_layers[i] = np.vectorize(lambda n: getattr(n, self.activation_function)())(self.hidden_layers[i])
                    print(type(self.hidden_layers[i][1]))
                    self.hidden_layers[i][0] = Neuron(1)
                elif i==self._weight_layer_count-1:
                    self._output_layer = np.sum(self.hidden_layers[i-1]*self.weight_layers[i].T,axis=1)
                    # self._output_layer = np.vectorize(lambda n: n.sigmoid())(self._output_layer)
                    self._output_layer = np.vectorize(lambda n: getattr(n, self.activation_function)())(self._output_layer)
                else:
                    self.hidden_layers[i] = np.sum(self.hidden_layers[i-1]*self.weight_layers[i].T,axis=1)
                    # self.hidden_layers[i] = np.vectorize(lambda n: n.sigmoid())(self.hidden_layers[i])
                    self.hidden_layers[i] = np.vectorize(lambda n: getattr(n, self.activation_function)())(self.hidden_layers[i])
                    self.hidden_layers[i][0] = Neuron(1)
            self.outputs.append(self._output_layer)
        else:
            raise ValueError(f"Weight layers not initialized, please initialize weight layers before feedforward")

    # TODO, implementasi yg ini
    def _backpropagation(self):
        raise NotImplementedError("Belum diimplement")

    def fit(self,X,Y):
        raise NotImplementedError("Belum diimplement")

    def predict(self,X):
        raise NotImplementedError("Belum diimplement")
    
    def mse(self, output, target):
        return np.sum(np.power(target-output, 2))/len(output)
    
    def binaryCrossEntropy(self, output, target):
        return -np.sum(target*np.log(output) + (np.subtract(1,target)*np.log(np.subtract(1, output))))/len(output)


In [729]:
modelInstance = FFNN19(
    input_neuron_count=2,
    hidden_layer_count=3,
    output_neuron_count=2,
    activation_function="sigmoid"
    )

In [730]:
modelInstance.setHiddenLayerNeuronCount([4,5,8])

In [731]:
modelInstance.uniformWeightDistribution(1,4,9)

In [732]:
modelInstance._feedforward([5,8])
print(1,5,7,"\n")

print(modelInstance.weight_layers[0],"\n")
print(modelInstance.hidden_layers[0],"\n")
print(type(modelInstance.hidden_layers[0][1]),"\n")
print(modelInstance.weight_layers[1],"\n")
print(modelInstance.hidden_layers[1],"\n")
print(modelInstance.weight_layers[2],"\n")
print(modelInstance.hidden_layers[2],"\n")
print(modelInstance.weight_layers[3],"\n")
print(modelInstance._output_layer)
print(modelInstance.binaryCrossEntropy(modelInstance._output_layer, [1, 2]))

# print(modelInstance.weight_layers)

<class '__main__.Neuron'>
1 5 7 

[[0 1.860451627262666 2.8094444501546856 3.3326022487605362
  3.148223888810696]
 [0 3.581180947548654 3.754712887146287 1.0797632034027917
  2.311744000388526]
 [0 1.1954625921103226 1.0168781520381787 3.491864310913771
  3.9499067328585267]] 

[1 1.000000000000183 1.0000000000001241 1.0000000000000002 1.0] 

<class '__main__.Neuron'> 

[[0 1.9468104699348194 3.116001113768361 1.8975432195947401
  3.222245448627246 1.8392416592448608]
 [0 3.96322224068373 3.9587156478407155 3.6485877009900274
  3.738430155351754 3.1246207423095176]
 [0 3.7697146242106614 1.2691923395629376 2.1103946354200223
  2.817816830527789 2.433189690756057]
 [0 1.5852591804371396 1.1867604922499901 1.3795552217197549
  3.888344754751488 1.3983236116882913]
 [0 2.131169112868656 2.8702649240223876 3.3462883920079083
  1.2359466813078437 3.7937123594291053]] 

[1 1.0000015209496809 1.0000041147416123 1.0000041918467484
 1.0000003371345019 1.0000034090119363] 

[[0 1.96289235317291

TypeError: loop of ufunc does not support argument 0 of type Neuron which has no callable log method

In [ ]:
print(modelInstance.outputs)
# print(modelInstance.outputs[1][0]._prev)


[array([1.0000000001938973, 1.000000000196997], dtype=object)]


In [ ]:


# n_input_neuron = 3 # jumlah neuron input
# n_input_instance = 1 # jumlah instance input
# n_hidden_layer = 2  # jumlah layer
# n_output_neuron = 1
# weight_layer_count = n_hidden_layer + 1


# input_layer_neuron = valueGenerator(np.random.randint,1,5,size=(n_input_instance,n_input_neuron))
# output_layer_neuron = valueGenerator(np.zeros,n_output_neuron)
# hidden_layer_neuron = [Value(0) for i in range(n_hidden_layer)]
# self.weight_layers = [Value(0) for i in range(weight_layer_count)]

# for i in range(n_hidden_layer):
#     hidden_layer_neuron[i] = np.zeros(int(input()))

# for i in range(weight_layer_count):
#     if i==0:
#         # weight_layer_neuron[i] = np.random.randint(5,size=(n_input_neuron,len(hidden_layer_neuron[i])))
#         self.weight_layers[i] = valueGenerator(np.random.randint,1,6,size=(n_input_neuron,len(hidden_layer_neuron[i])))
#     elif i==weight_layer_count-1:
#         self.weight_layers[i] = valueGenerator(np.random.randint,1,6,size=(len(hidden_layer_neuron[i-1]),n_output_neuron))
#         # weight_layer_neuron[i] = np.random.randint(5,size=(len(hidden_layer_neuron[i-1]),n_output_neuron))
#     else:
#         self.weight_layers[i] = valueGenerator(np.random.randint,1,6,size=(len(hidden_layer_neuron[i-1]),len(hidden_layer_neuron[i])))
#         # weight_layer_neuron[i] = np.random.randint(5,size=(len(hidden_layer_neuron[i-1]),len(hidden_layer_neuron[i])))

# for i in range(weight_layer_count):
#     if i==0:
#         hidden_layer_neuron[i] = np.sum(input_layer_neuron[0]*self.weight_layers[i].T,axis=1)
#     elif i==weight_layer_count-1:
#         output_layer_neuron = np.sum(hidden_layer_neuron[i-1]*self.weight_layers[i].T,axis=1)
#     else:
#         hidden_layer_neuron[i] = np.sum(hidden_layer_neuron[i-1]*self.weight_layers[i].T,axis=1)

# print("Input")
# print(input_layer_neuron)
# print("Hidden Layer")
# print(hidden_layer_neuron)
# print("Output")
# print(output_layer_neuron)
# print("Weight")
# print(self.weight_layers)
# print("\n")

# print(type(self.weight_layers[0][0][0]))